<a href="https://colab.research.google.com/github/ftk1000/5G/blob/master/parse_3gpp_v2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parse 3GPP document
2020.08.19

parse_3gpp_v2.ipynb

In [3]:
# -*- coding: utf-8 -*-
# parse_3gpp_v2.py
# 2020.08.19
#---------------------------------------------
#  README:
#---------------------------------------------
#prepare data set : parse 3GPP 36523-1-f00_s07_01.txt and 
#extract sentences corresponding to "test case name", "with", "when", "then"
#
# SOURCE DOCUMENT
# LTE; Evolved Universal Terrestrial Radio Access (E-UTRA) and Evolved Packet Core (EPC);
# User Equipment (UE) conformance specification; Part 1: Protocol conformance specification
# 3GPP TS 36.523-1

# downloaded *.DOC files are here:  .\36523-1-f00.zip
# we will use only of them converted to TXT format#
#INPUT DATA:
#.\36523-1-f00_s07_01.txt
#
#36523-1-f00_s07_01.txt files was generated from 
#36523-1-f00_s07_01.DOC file located in
#
#OUTPUT:
#.\36523-1-f00_s07_01_tests.csv   

In [4]:
### Here are the instructions how to read a file from Gdrive
# https://buomsoo-kim.github.io/colab/2018/04/16/Importing-files-from-Google-Drive-in-Google-Colab.md/

# SOURCE DOCUMENT
# LTE; Evolved Universal Terrestrial Radio Access (E-UTRA) and Evolved Packet Core (EPC);
# User Equipment (UE) conformance specification; Part 1: Protocol conformance specification
# 3GPP TS 36.523-1

# downloaded *.DOC files are here:  C:\Users\khafifa\Downloads\36523-1-f00.zip
# we will use only of them converted to TXT format
file_name = wd+'36523-1-f00_s07_01.txt'

# shareble_link = 'https://drive.google.com/file/d/1jNGAf5O6U4Cr4GbWK_DpCxIuxv7-BQ2c/view?usp=sharing';  file_id = '1jNGAf5O6U4Cr4GbWK_DpCxIuxv7-BQ2c'
shareble_link = 'https://drive.google.com/file/d/0B1Iyay9XxksvMG9wLUpFQm1Hbnc/view?usp=sharing';  file_id = '0B1Iyay9XxksvMG9wLUpFQm1Hbnc'

!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate, create the PyDrive client, and READ DATA

In [ ]:
#see Step 6 https://buomsoo-kim.github.io/colab/2018/04/16/Importing-files-from-Google-Drive-in-Google-Colab.md/
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':file_id})   # replace the id with id of file you want to access
downloaded.GetContentFile('36523-1-f00_s07_01.txt') 

In [ ]:
# to deal with non-ascii codes read this
# https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s
# import codecs      # f = codecs.open(file_name, "r", encoding= 'unicode_escape')
file_name = '36523-1-f00_s07_01.txt'
with open(file=file_name, mode='r', encoding='unicode_escape') as f:
    lines = f.readlines()
f.close()
len(lines)

102056

In [ ]:
import numpy as np
import pandas as pd
import string
import random

wd = 'C:\\Users\\Farid Khafizov\\Downloads\\'
file_name = '36523-1-f00_s07_01.txt'

with open(file=wd+file_name, mode='r', encoding='unicode_escape') as f:
    lines = f.readlines()
f.close()
print(len(lines))
#%%
printable = set(string.printable)
lines2 = [  ''.join(filter(lambda x: x in printable, s)) for s in lines  ]
print(len(lines2))

# TAKE A LOOK AT DATA

In [ ]:
# Show a few lines describing a test case
k=218
lines[k-4:k+10]

['\n',
 '7.1.1.1.3.3\tSpecific message contents\n',
 'None.\n',
 '7.1.1.1a\tCCCH mapped to UL SCH/ DL-SCH / UE Cat 0\n',
 '7.1.1.1a.1\tTest Purpose (TP)\n',
 '(1)\n',
 'with { UE in E-UTRA RRC_IDLE state }\n',
 'ensure that {\n',
 '  when { UE receives a Paging message including a matched identity }\n',
 '    then { UE responds with a RRCConnectionRequest message in a MAC PDU on UL SCH on CCCH indicating LCID \x9201011\x92  }\n',
 '            }\n',
 '\n',
 '7.1.1.1a.2\tConformance requirements\n',
 'References: The conformance requirements covered in the current TC are specified in: TS 36.321, clause 5.3.3, 5.11, 6.1.2 and 6.2.1.\n']

In [ ]:
# Some 'with' are followed by '{', some by '('
print(lines[220].strip())
print(lines[16000].strip())

with { UE in E-UTRA RRC_IDLE state }
with ( UE in E-UTRA RRC_CONNECTED state )


# PARSING/SEARCH/REGEX FUNCTIONS

In [ ]:
# find line numbers that contain PATTERN
def find_lineNums(lines, pattern):
    test_idx = []
    for i in range(len(lines)):
      line = lines[i].strip();  # print(line)
      if line.find(pattern) >=0 :
        test_idx.append(i)
    return test_idx

PATTERN = "Test Purpose (TP)"
test_idx = find_lineNums(lines=lines, pattern= PATTERN)    
print(test_idx)
print("patterns found =",len(test_idx))

[36, 218, 352, 508, 873, 1087, 1243, 1596, 1949, 2241, 2621, 2782, 2973, 3196, 3435, 3725, 4275, 5016, 5605, 5754, 6360, 6621, 6707, 6933, 7193, 7507, 7807, 8127, 8250, 8544, 9076, 9544, 10221, 10660, 11099, 11743, 12380, 12620, 13509, 13656, 14067, 14273, 15027, 15295, 15383, 15532, 15998, 16568, 16809, 17228, 17548, 17768, 18205, 18605, 18918, 19220, 19883, 20194, 20455, 20758, 21173, 21646, 21989, 22399, 22661, 23370, 23385, 23748, 24075, 24312, 24649, 24963, 25424, 25460, 25765, 26315, 26532, 26903, 27753, 27920, 28137, 28320, 28428, 28535, 28926, 29245, 29564, 29878, 30213, 30548, 34821, 38947, 43126, 47278, 51785, 56037, 61926, 67310, 72686, 78107, 83529, 84824, 86116, 92189, 93231, 94224, 98418, 99255, 99901, 100314, 100705, 101044, 101333, 101881]
patterns found = 114


In [ ]:
import re
# search for "(4)", "(12)", etc
R = re.compile(r"^\(\d+\)")

# keep looking for '(\d)' until find END='Conformance requirements'
# save line number for each match including the END 
def find_idx_subtests(ix1,ix2):
  ix_subtest = []
  for i in range(ix1,ix2):
    M = R.search(lines[i])
    if M:
      ix_subtest.append(i)
      # print('==> lineNum =',i, "  line=", lines[i].strip())
    if lines[i].find("Conformance requirements") >=0 :
        ix_subtest.append(i)   
        break;
  return ix_subtest


# test
k=46
ix1=test_idx[k]; ix2=test_idx[k+1]
subtests = find_idx_subtests(ix1, ix2)  
for p in range(len(subtests)):
    print(subtests[p], '    ',lines[subtests[p]].strip() )  

15999      (1)
16006      (2)
16013      (3)
16020      (4)
16026      (5)
16033      (6)
16039      (7)
16045      (8)
16052      7.1.4.6.2	Conformance requirements


In [ ]:
import pandas as pd
d = pd.DataFrame(columns=['doc','test_sec', 'test_name', 'test_num',  'with', 'when', 'then'])
doc_name = file_name.split('.')[0]

In [ ]:
#%%  BUILD THE DATA BASE
df_index = 0

for k in range(len(test_idx)-1):
    ix, ix2 =test_idx[k], test_idx[k+1] # get two test case locations
    [sec_num, test_name] = lines[ix-1].strip().split('\t')
    ix_subtests = find_idx_subtests(ix,ix2)
    for p in range(len(ix_subtests)-1):
        index = ix_subtests[p]
        
        with_kw = when_kw = then_kw = ''
        while index < ix_subtests[p+1]:
            line = lines[index].strip()
            if line.find('with') >=0 :
                with_kw = line[len('with'):].strip()[1:-1].strip()
            if line.find('when') >=0 :
                when_kw = line[len('when'):].strip()[1:-1].strip()
            if line.find('then') >=0 :
                then_kw = line[len('then'):].strip()[1:-1].strip()
            index += 1
            
        d.loc[df_index] = [doc_name, sec_num,  test_name, p+1, with_kw, when_kw, then_kw] 
        df_index += 1


In [ ]:
print(d.shape)
d.head()

(279, 7)


,doc,test_sec,test_name,test_num,with,when,then
0,36523-1-f00_s07_01,7.1.1.1,CCCH mapped to UL SCH/ DL-SCH / Reserved Logic...,1,UE in E-UTRA RRC_IDLE state and after transmit...,UE receives a MAC PDU on DL SCH and addressed ...,UE discards the MAC PDU
1,36523-1-f00_s07_01,7.1.1.1,CCCH mapped to UL SCH/ DL-SCH / Reserved Logic...,2,UE receives a MAC PDU on DL SCH and addressed ...,UE receives a MAC PDU on DL SCH and addressed ...,UE forwards to upper layers the disassembled a...
2,36523-1-f00_s07_01,7.1.1.1a,CCCH mapped to UL SCH/ DL-SCH / UE Cat 0,1,UE responds with a RRCConnectionRequest messag...,UE receives a Paging message including a match...,UE responds with a RRCConnectionRequest messag...
3,36523-1-f00_s07_01,7.1.1.2,DTCH or DCCH mapped to UL SCH/ DL-SCH / Reserv...,1,UE in E-UTRA RRC_Connected state with DRB [Log...,UE receives a MAC PDU on DL SCH and addressed ...,UE shall not forward the disassembled and demu...
4,36523-1-f00_s07_01,7.1.1.2,DTCH or DCCH mapped to UL SCH/ DL-SCH / Reserv...,2,UE receives a MAC PDU on DL SCH and addressed ...,UE receives a MAC PDU on DL SCH and addressed ...,UE shall forward the disassembled and demultip...


In [ ]:
wd='./'
d.to_csv(wd+doc_name+'_tests.csv')

 # GENERATE TRAIN AND TEST SETS

In [ ]:
#%% GENERATE TRAIN AND TEST SETS
import numpy as np
txt = np.array(list(d['with'])    + list(d['when'])    + list(d['then']))
lbl = np.array([0]*len(d['with']) + [1]*len(d['when']) + [2]*len(d['then']) )

assert len(lbl)==len(txt)

# Shuffle and split
idx = np.random.permutation(len(txt))
x,y = txt[idx], lbl[idx]
df=pd.DataFrame({'sentence':x,'label':y})

In [ ]:
df.head()

,sentence,label
0,UE starts the drx-ULRetransmissionTimer for th...,2
1,UE in E-UTRA RRC_CONNECTED state,0
2,UE decodes the received transport block of siz...,2
3,UE in E-UTRA RRC_IDLE state and after transmit...,0
4,UE in enhanced coverage receives a Paging mess...,1
